In [2]:
# ENPM 661 - Planning for Autonomous Robots
# Project 3 - Implementing A* Algorithm for Turtlebot 3
# Team - Nidhi Bhojak           , UID - 116787529
#        Kulbir Singh Ahluwalia , UID - 116836050
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
from time import process_time


def cartesian_to_image_coord(adjust_coord):
    return [adjust_coord[0], 200 - adjust_coord[1]]

def find_line_slope_and_intercept(test_point_coord, line_point_1, line_point_2):
    slope = (line_point_2[1] - line_point_1[1]) / (line_point_2[0] - line_point_1[0])
    intercept = line_point_1[1] - (slope * line_point_1[0])
    # print(slope,intercept)
    return slope, intercept

# function returns false when the point is outside the circle
def circular_obstacle(clearance, radius_rigid_robot, test_point_coord):
    circle_center = (225, 150)
    test_point_coord_x = test_point_coord[0]
    test_point_coord_y = test_point_coord[1]
    augment_distance = radius_rigid_robot + clearance

    distance_from_center = ((test_point_coord_x - circle_center[0]) ** 2 + (
                test_point_coord_y - circle_center[1]) ** 2) ** 0.5

    if distance_from_center > (25 + augment_distance):
        return False
    else:
        return True

# function returns false when the point is outside the ellipse
def ellipsoid_obstacle(clearance, radius_rigid_robot, test_point_coord):
    ellipsoid_center = (150, 100)
    test_point_coord_x = test_point_coord[0]
    test_point_coord_y = test_point_coord[1]
    augment_distance = radius_rigid_robot + clearance
    semi_major_axis = 40
    semi_minor_axis = 20

    distance_from_center = ((test_point_coord_x - ellipsoid_center[0]) ** 2) / (
                (semi_major_axis + augment_distance) ** 2) + (test_point_coord_y - ellipsoid_center[1]) ** 2 / (
                                       (semi_minor_axis + augment_distance) ** 2)

    if distance_from_center > 1:
        return False
    else:
        return True

def rectangle_obstacle(clearance, radius_rigid_robot, test_point_coord):
    circle_center = (225, 150)
    augment_distance = radius_rigid_robot + clearance

    rectangle_point_1 = [100, 38.66025]
    rectangle_point_2 = [35.0481, 76.1603]
    rectangle_point_3 = [30.0481, 67.5]
    rectangle_point_4 = [95, 30]

    # We set the flags by testing for image point inside the rectangle
    # Because the sign for the half plane is unique for every line, we test it by using image point that is confirmed to be inside the rectangle
    edge1_m_c = find_line_slope_and_intercept(test_point_coord, rectangle_point_1, rectangle_point_2)
    line1 = test_point_coord[1] - (edge1_m_c[0] * test_point_coord[0]) - (
                edge1_m_c[1] + (augment_distance * 2 / (3 ** 0.5)))
    # print(line1)
    if line1 >= 0:
        flag1 = False
        # print("False")
    else:
        flag1 = True
        # print("True")

    edge2_m_c = find_line_slope_and_intercept(test_point_coord, rectangle_point_2, rectangle_point_3)
    line2 = test_point_coord[1] - (edge2_m_c[0] * test_point_coord[0]) - (edge2_m_c[1] + (augment_distance * 2))
    # print(line2)
    if line2 >= 0:
        flag2 = False
        # print("False")
    else:
        flag2 = True
        # print("True")

    edge3_m_c = find_line_slope_and_intercept(test_point_coord, rectangle_point_3, rectangle_point_4)
    line3 = test_point_coord[1] - (edge3_m_c[0] * test_point_coord[0]) - (
                edge3_m_c[1] - (augment_distance * 2 / (3 ** 0.5)))
    # print(line3)
    if line3 >= 0:
        flag3 = True
        # print("True")
    else:
        flag3 = False
        # print("False")

    edge4_m_c = find_line_slope_and_intercept(test_point_coord, rectangle_point_4, rectangle_point_1)
    line4 = test_point_coord[1] - (edge4_m_c[0] * test_point_coord[0]) - (edge4_m_c[1] - (augment_distance * 2))
    # print(line4)
    if line4 >= 0:
        flag4 = True
        # print("True")
    else:
        flag4 = False
        # print("False")

    if flag1 and flag2 and flag3 and flag4:
        return True
    else:
        return False

def rhombus_obstacle(clearance, radius_rigid_robot, test_point_coord):
    augment_distance = radius_rigid_robot + clearance

    rhombus_point_1 = [250, 25]
    rhombus_point_2 = [225, 40]
    rhombus_point_3 = [200, 25]
    rhombus_point_4 = [225, 10]

    # We set the flags by testing for image point inside the rectangle
    # Because the sign for the half plane is unique for every line, we test it by using image point that is confirmed to be inside the rectangle
    edge1_m_c = find_line_slope_and_intercept(test_point_coord, rhombus_point_1, rhombus_point_2)
    line1 = test_point_coord[1] - (edge1_m_c[0] * test_point_coord[0]) - (edge1_m_c[1] + (augment_distance / 0.8575))
    # print(line1)
    if line1 >= 0:
        flag1 = False
    else:
        flag1 = True

    edge2_m_c = find_line_slope_and_intercept(test_point_coord, rhombus_point_2, rhombus_point_3)
    line2 = test_point_coord[1] - (edge2_m_c[0] * test_point_coord[0]) - (edge2_m_c[1] + (augment_distance / 0.8575))
    # print(line2)
    if line2 >= 0:
        flag2 = False
    else:
        flag2 = True

    edge3_m_c = find_line_slope_and_intercept(test_point_coord, rhombus_point_3, rhombus_point_4)
    line3 = test_point_coord[1] - (edge3_m_c[0] * test_point_coord[0]) - (edge3_m_c[1] - (augment_distance / 0.8575))
    # print(line3)
    if line3 >= 0:
        flag3 = True
    else:
        flag3 = False

    edge4_m_c = find_line_slope_and_intercept(test_point_coord, rhombus_point_4, rhombus_point_1)
    line4 = test_point_coord[1] - (edge4_m_c[0] * test_point_coord[0]) - (edge4_m_c[1] - (augment_distance / 0.8575))
    # print(line4)
    if line4 >= 0:
        flag4 = True
    else:
        flag4 = False

    if flag1 and flag2 and flag3 and flag4:
        return True
    else:
        return False

def nonconvex_obstacle_right_half(clearance, radius_rigid_robot, test_point_coord):
    augment_distance = radius_rigid_robot + clearance

    nonconvex_point_1 = [100, 150]
    nonconvex_point_2 = [75, 185]
    nonconvex_point_3 = [60, 185]
    nonconvex_point_4 = [50, 150]
    nonconvex_point_5 = [75, 120]

    # We set the flags by testing for image point inside the rectangle
    # Because the sign for the half plane is unique for every line, we test it by using image point that is confirmed to be inside the nonconvex_obstacle
    edge1_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_1, nonconvex_point_2)
    line1 = test_point_coord[1] - (edge1_m_c[0] * test_point_coord[0]) - (edge1_m_c[1] + (augment_distance / 0.58124))
    # print(line1)
    if line1 >= 0:
        flag1 = False
        # print("False")
    else:
        flag1 = True
        # print("True")

    edge2_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_2, nonconvex_point_3)
    line2 = test_point_coord[1] - (edge2_m_c[0] * test_point_coord[0]) - (edge2_m_c[1] + (augment_distance / 1))
    # print(line2)
    if line2 >= 0:
        flag2 = False
        # print("False")
    else:
        flag2 = True
        # print("True")

    # edge 3 is not augmented with clearance+robot_radius since its inside the nonconvex polygon
    edge3_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_3, nonconvex_point_4)
    line3 = test_point_coord[1] - (edge3_m_c[0] * test_point_coord[0]) - (edge3_m_c[1] + (augment_distance / 0.27472))
    # print(line3)
    if line3 >= 0:
        flag3 = False
        # print("False")
    else:
        flag3 = True
        # print("True")

    edge4_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_4, nonconvex_point_5)
    line4 = test_point_coord[1] - (edge4_m_c[0] * test_point_coord[0]) - (edge4_m_c[1] - (augment_distance / 0.64018))
    # print(line4)
    if line4 >= 0:
        flag4 = True
        # print("True")
    else:
        flag4 = False
        # print("False")

    edge5_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_5, nonconvex_point_1)
    line5 = test_point_coord[1] - (edge5_m_c[0] * test_point_coord[0]) - (edge5_m_c[1] - (augment_distance / 0.640184))
    # print(line4)
    if line5 >= 0:
        flag5 = True
        # print("True")
    else:
        flag5 = False
        # print("False")

    if flag1 and flag2 and flag3 and flag4 and flag5:
        return True
    else:
        return False

def nonconvex_obstacle_left_half(clearance, radius_rigid_robot, test_point_coord):
    augment_distance = radius_rigid_robot + clearance

    nonconvex_point_1 = [50, 150]
    nonconvex_point_2 = [60, 185]
    nonconvex_point_3 = [25, 185]
    nonconvex_point_4 = [20, 120]

    # We set the flags by testing for image point inside the rectangle
    # Because the sign for the half plane is unique for every line, we test it by using image point that is confirmed to be inside the nonconvex_obstacle
    edge1_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_1, nonconvex_point_2)
    line1 = test_point_coord[1] - (edge1_m_c[0] * test_point_coord[0]) - (edge1_m_c[1] - (augment_distance / 0.27472))
    # print(line1)
    if line1 >= 0:
        flag1 = True
        # print("True")
    else:
        flag1 = False
        # print("False")

    edge2_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_2, nonconvex_point_3)
    line2 = test_point_coord[1] - (edge2_m_c[0] * test_point_coord[0]) - (edge2_m_c[1] + (augment_distance / 1))
    # print(line2)
    if line2 >= 0:
        flag2 = False
        # print("False")
    else:
        flag2 = True
        # print("True")

    # edge 3 is not augmented with clearance+robot_radius since its inside the nonconvex polygon
    edge3_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_3, nonconvex_point_4)
    line3 = test_point_coord[1] - (edge3_m_c[0] * test_point_coord[0]) - (edge3_m_c[1] + (augment_distance / 0.0767))
    # print(line3)
    if line3 >= 0:
        flag3 = False
        # print("False")
    else:
        flag3 = True
        # print("True")

    edge4_m_c = find_line_slope_and_intercept(test_point_coord, nonconvex_point_4, nonconvex_point_1)
    line4 = test_point_coord[1] - (edge4_m_c[0] * test_point_coord[0]) - (edge4_m_c[1] - (augment_distance / 0.7071))
    # print(line4)
    if line4 >= 0:
        flag4 = True
        # print("True")
    else:
        flag4 = False
        # print("False")

    if flag1 and flag2 and flag3 and flag4:
        return True
    else:
        return False

def boundary_obstacle(clearance, radius_rigid_robot, test_point_coord):
    augment_distance = radius_rigid_robot + clearance
    x = test_point_coord[0]
    y = test_point_coord[1]

    if 0 <= x < augment_distance:
        return True
    elif (299 - augment_distance) < x <= 299:
        return True
    elif 0 <= y < augment_distance:
        return True
    elif (199 - augment_distance) < y <= 199:
        return True
    else:
        return False

class GraphNode:
    def __init__(self, point):
        self.position = point
        self.cost = math.inf
        self.parent = None

# THe following code outputs True if the point is inside the obstacle or False if outside
def test_point_obstacle_check(clearance, radius_rigid_robot, test_point_coord):
    test_point_coord = cartesian_to_image_coord(test_point_coord)
    if circular_obstacle(clearance, radius_rigid_robot, test_point_coord):
        return True
    elif ellipsoid_obstacle(clearance, radius_rigid_robot, test_point_coord):
        return True
    elif rectangle_obstacle(clearance, radius_rigid_robot, test_point_coord):
        return True
    elif rhombus_obstacle(clearance, radius_rigid_robot, test_point_coord):
        return True
    elif nonconvex_obstacle_right_half(clearance, radius_rigid_robot, test_point_coord):
        return True
    elif nonconvex_obstacle_left_half(clearance, radius_rigid_robot, test_point_coord):
        return True
    elif boundary_obstacle(clearance, radius_rigid_robot, test_point_coord):
        return True
    else:
        return False



In [3]:
# for testing
start_node_x = 5
start_node_y = 10
goal_node_x = 20
goal_node_y = 30
clearance = 2
radius_rigid_robot = 2

print(test_point_obstacle_check(clearance, radius_rigid_robot,[230,40]))



True


In [7]:
print(test_point_obstacle_check(clearance, radius_rigid_robot,[5,5]))
rounded_till_first_decimal = np.around([0.37, 1.64], decimals=1)
print(list(rounded_till_first_decimal))



False
[0.4, 1.6]


In [5]:



def round_off_till_threshold(number):
    number_double = round((number * 2))
    return number_double/2



In [52]:
round_off(2.4)


2.5

In [8]:
def get_minimum_element(queue):
    min_index = 0
    for index in range(len(queue)):
        if queue[index].cost < queue[min_index].cost:
            min_index = index
    return queue.pop(min_index)



In [9]:
get_minimum_element([1,2,3,4,5,6,7,8])



AttributeError: 'int' object has no attribute 'cost'

In [3]:
import math
def heu(node1, node2):
  dist= math.sqrt( (node1[0] - node2[0])**2 + (node1[1] - node2[1])**2)
  return dist



In [5]:
heu([0,0],[3,4])



5.0

In [7]:
import numpy as np
line1 = [2,3]
line2 = [1,1]
b = [5, 1]
np.linalg.solve([line1,line2], b)


array([-2.,  3.])

In [5]:
from sympy import symbols, Eq, solve
y = symbols('y')
eq1 = Eq(y**2 - 5*y + 6, 0)   #solutions are y=3, y=2



In [26]:

def find_line_slope_and_intercept(test_point_coord, line_point_1, line_point_2):
    slope = (line_point_2[1] - line_point_1[1]) / (line_point_2[0] - line_point_1[0])
    intercept = line_point_1[1] - (slope * line_point_1[0])
    # print(slope,intercept)
    return slope, intercept




In [22]:
from sympy import solve, Poly, Eq, Function, exp
from sympy.abc import x, y, z, a, b
# f = Function('f')
equation_1 = x**2 - 3
equation_2 = y - 1
# solution = solve([equation_1, equation_2], dict=True)
solution2 = solve([equation_1, equation_2], (x,y))
print(solution2)

#print solutions as dictionaries
print(solution[0])
print(solution[1])

# get the solutions from the dictionaries using the keys "x" and "y"
print("x1 is ", solution[0][x])
print("y1 is ", solution[0][y])

print("x2 is ", solution[1][x])
print("y2 is ", solution[1][y])

[(-sqrt(3), 1), (sqrt(3), 1)]
{x: -sqrt(3), y: 1}
{x: sqrt(3), y: 1}
x1 is  -sqrt(3)
y1 is  1
x2 is  sqrt(3)
y2 is  1


In [27]:
from sympy import solve, Poly, Eq, Function, exp
from sympy.abc import x, y, z, a, b
equation_1 = x**2 - 3
equation_2 = y - 1
#the following equation will output solutions in the form of a list
solution2 = solve([equation_1, equation_2], (x,y))
print(solution2)


[(-sqrt(3), 1), (sqrt(3), 1)]


In [32]:
def circular_intersection_check(clearance, radius_rigid_robot, parent_coord, child_coord):
    
    augment_distance = radius_rigid_robot + clearance
    
    line_m_c = find_line_slope_and_intercept(None, child_coord, parent_coord)
    line_equation = y - (line_m_c[0] * x) - (line_m_c[1])
    print(line_equation)
    
    circle_center = (225, 150)
    equation_1 = (x-225)**2 + (y-150)**2 - (25+augment_distance)**2
    equation_2 = line_equation
    #the following equation will output solutions in the form of a list
    solution2 = solve([equation_1, equation_2], (x,y))
    print(solution2)

In [33]:
circular_intersection_check(1,1,[200,150],[250,150])



y - 150.0
[(198.000000000000, 150.000000000000), (252.000000000000, 150.000000000000)]
